In [61]:
import os
import cv2
import yaml
import shutil
import random
from tqdm import tqdm
from pathlib import Path
from ultralytics import YOLO


In [62]:
IMG_PATH = Path(r'./data/img_dataset')
train_val_test = (0.8,0.1,0.1)
ANNOTATION_PATH = Path(r'./data/annotation')
YOLO_ANNOTATION = Path(r'./data/yolo_annotation')

class_dict = {category_name: idx for idx, category_name in enumerate(os.listdir(IMG_PATH))}
print(class_dict)

{'People': 0, 'Motorbike': 1, 'Cup': 2, 'Bicycle': 3, 'Chair': 4, 'Boat': 5, 'Table': 6, 'Car': 7, 'Bottle': 8, 'Bus': 9, 'Cat': 10, 'Dog': 11}


In [63]:
def create_data_yaml(path='./'):
    data = {
        'path': path,
        'train': './data/train_data',
        'val': './data/val_data',
        'nc': len(class_dict),
        'names': list(class_dict.keys()),
    }

    with open('data.yaml', 'w') as f:
        yaml.dump(data, f, default_flow_style=False)

# Example usage
create_data_yaml()

In [64]:
def corresponding_image_path(folder_path, annotaion_path):
    file_title, _ = os.path.splitext(annotaion_path)
    return IMG_PATH / folder_path / file_title

def pascal_to_yolo():
    for sub_category in os.listdir(ANNOTATION_PATH):
        folder_path = ANNOTATION_PATH / sub_category
        for file_name in os.listdir(folder_path):
            file_path = folder_path / file_name
            
            yolo_annotation_path = YOLO_ANNOTATION/sub_category/file_name
            if os.path.isfile(yolo_annotation_path):
                continue

            os.makedirs(YOLO_ANNOTATION/sub_category, exist_ok=True)

            image_path = corresponding_image_path(sub_category, file_name)
            img = cv2.imread(image_path)
            if img is None:
                print(f"{file_path} cannot be read.")
                continue

            img_width, img_height = img.shape[:2]

            with open(file_path, 'r') as anno_file:
                pascal_data = anno_file.readlines()[1:]

            pascal_num = list(map(lambda s: s.strip().split(" ")[:5], pascal_data))
            
            yolo_format = []
            for indv_bounding_coordinates in pascal_num:
                indv_bounding_coordinates[0] = class_dict[indv_bounding_coordinates[0]]
                indv_bounding_coordinates[1:] = list(map(int, indv_bounding_coordinates[1:]))

                cv2.rectangle(img, 
                              (indv_bounding_coordinates[1], indv_bounding_coordinates[2]), 
                              (indv_bounding_coordinates[1]+indv_bounding_coordinates[3],indv_bounding_coordinates[2]+indv_bounding_coordinates[4]),
                              color=(0,200,0),
                              thickness=2
                              )
                

                class_category = indv_bounding_coordinates[0]
                x_ctr = (indv_bounding_coordinates[1] + indv_bounding_coordinates[3]/2) / img_width
                y_ctr = (indv_bounding_coordinates[2] + indv_bounding_coordinates[4]/2) / img_height
                width = (indv_bounding_coordinates[3]) / img_width
                height = (indv_bounding_coordinates[4]) / img_height

                yolo_format.append([class_category, x_ctr, y_ctr, width, height])
            
            # cv2.imshow(str(file_path), img)
            # cv2.waitKey(0)

            with open(yolo_annotation_path, 'w') as yolo_writer:
                for data in yolo_format:
                    yolo_writer.write(f"{' '.join(list(map(str, data)))}\n")
                
            

pascal_to_yolo()

data/annotation/Bicycle/2015_00391.jpg.txt cannot be read.


[ WARN:0@3029.729] global loadsave.cpp:268 findDecoder imread_('data/img_dataset/Bicycle/2015_00391.jpg'): can't open/read file: check file path/integrity


In [65]:
with open("data.yaml") as f:
    cfg = yaml.safe_load(f)

print(cfg)

{'names': ['People', 'Motorbike', 'Cup', 'Bicycle', 'Chair', 'Boat', 'Table', 'Car', 'Bottle', 'Bus', 'Cat', 'Dog'], 'nc': 12, 'path': './', 'train': './data/train_data', 'val': './data/val_data'}


In [66]:
model = YOLO('./snapshots/yolov8n.pt')

print(model.info())

YOLOv8n summary: 129 layers, 3,157,200 parameters, 0 gradients, 8.9 GFLOPs
(129, 3157200, 0, 8.8575488)


In [67]:
config_settings = {
    # Output Config
    "project": "./YOLOv8_outputs/runs/train",
    "name": "yolov8_exp",

    # Train Config
    "epochs": 1000,
    "patience": 10,
    "batch": -1,
    "imgsz": 640,
    "save": True,
    "save_period": 1,
    "cache": False,
    "device": 0,
    "workers": 8,
    "project": "runs/train",
    "name": "yolov8_exp",
    "exist_ok": True,
    "pretrained": True,
    "optimizer": "auto",
    "deterministic": False,
    "classes":list(class_dict.keys()),
    "rect": False,
    "resume": True,
    "visualize": True,

    # "lr0": 0.001,
    # "lrf": 0.01,
    # "momentum": 0.937,
    # "weight_decay": 0.0005,
    # "warmup_epochs": 3.0,
    # "warmup_momentum": 0.8,
    # "warmup_bias_lr": 0.1,
    # "box": 7.5,
    # "cls": 0.5,
    # "dfl": 1.5,

    # Augmentation Config
    "hsv_h": 0.015,
    "hsv_s": 0.7,
    "hsv_v": 0.4,
    "degrees": 0.0,
    "translate": 0.1,
    "scale": 0.5,
    "shear": 0.0,
    "perspective": 0.0,
    "flipud": 0.0,
    "fliplr": 0.5,
    "mosaic": 1.0,
    "mixup": 0.0,
    "copy_paste": 0.0,
    "dropout": 0.0,

}


In [68]:
model.train(
    data="data.yaml",
    **config_settings
)


Ultralytics 8.3.107 🚀 Python-3.10.17 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce GTX 1080, 8106MiB)
engine/trainer: task=detect, mode=train, model=snapshots/yolov8n.pt, data=data.yaml, epochs=500, time=None, patience=50, batch=-1, imgsz=640, save=True, save_period=-1, cache=disk, device=0, workers=8, project=YOLOv8, name=yolov8n, exist_ok=False, pretrained=False, optimizer=SGD, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=snapshots/yolov8n.pt, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=False, val=True, split=val, save_json=False, conf=0.001, iou=0.7, max_det=300, half=True, dnn=False, plots=True, source=ultralytics/assets/, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show

RuntimeError: Dataset 'data.yaml' error ❌ 
Dataset 'data.yaml' images not found ⚠️, missing path '/home/yc-family/Documents/code/mlgp_new/datasets/data/val_data'
Note dataset download directory is '/home/yc-family/Documents/code/mlgp_new/datasets'. You can update this in '/home/yc-family/.config/Ultralytics/settings.json'